In [ ]:
# Подготовка
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, roc_curve

import zipfile


__z = zipfile.ZipFile("dataset.zip")
#df_str = __z.open("Vehicle_policies_2020.csv").read().decode("utf-8")
df_str_fs = __z.open("car_price_prediction.csv")

df_src = pd.read_csv(df_str_fs)


# Предварительная обработка данных

Данные [отсюда](https://www.kaggle.com/datasets/deepcontractor/car-price-prediction-challenge):

~~1. ID: индификатор продажи. ~~

2. Price: цена машины (этот столбец будем предсказывать). 

3. Levy: налог. 

4. Manufacturer: производитель. 

5. Model: модель машины. 

6. Prod. year: год производства. 

7. Category: тип машины. 

8. Leather interior: есть ли кожанный салон. 

9. Fuel type: тип топлива/тип двигателя. 

10. Engine volume: рабочий объём двигателя. 

11. Mileage: пробег. 

12. Cylinders: количество цилиндров. 

13. Gear box type: тип коробки передач. 

14. Drive wheels: ведущее(ие) колес(а). 

15. Doors: двери. 

16. Wheel: расположение руля (слева или справа). 

17. Color: цвет. 

18. Airbags: количество подушек безопасности. 


In [ ]:
# Look dataframe
pd.set_option('display.max_columns', df_src.shape[1])
df_src.head(300)

In [ ]:
# Look dataframe
#df_src.head()
pd.set_option('display.max_columns', df_src.shape[1])
df_src.tail(300)

# Проектирование признаков

## Убираем дубликаты

In [ ]:
print(f"Number of (rows, columns): {df_src.shape}")
duplicate_rows_df = df_src[df_src.duplicated()]
print(f"Number of duplicate (rows, columns): {duplicate_rows_df.shape}")
df_src = df_src.drop_duplicates()
print(f"Number of (rows, columns) after drop dublicates: {df_src.shape}")

## Смотрим, чтобы не было нерелевантных данных

### Как поступить с такими данными?

![](./imgs/if_missing_data.png)


In [ ]:
pd.set_option('display.max_rows', df_src.shape[1])
df_src.count() # Кол-во не None значений в каждой колонке

In [ ]:
pd.set_option('display.max_rows', df_src.shape[1])
print(df_src.isnull().sum()) # Смотрим есть ли null хотя бы в каком-нибудь столбце


In [ ]:
plt.figure(figsize=(100,50))

print((df_src["Levy"] == "-").value_counts())
print((df_src["Levy"] == "-").value_counts(normalize=True))

(df_src["Levy"]).value_counts().plot(kind='bar', subplots=True)


levy_fit_true = df_src[(df_src["Levy"] != "-") & (df_src["Model"] == "FIT") & (df_src["Manufacturer"] == "HONDA")].shape[0]

levy_fit_false = df_src[(df_src["Levy"] == "-") & (df_src["Model"] == "FIT") & (df_src["Manufacturer"] == "HONDA")].shape[0]

print(f"\n\tFor example if levy exists for HONDA FIT: {levy_fit_true} peaces. ")
print(f"\n\tFor example if levy not exists for HONDA FIT: {levy_fit_false} peaces. ")

# df_src.hist(column="Levy")

# print(f"Range of Levy: [{df_src['Levy'].min()} - {df_src['Levy'].max()}]")


In [ ]:
IF_Levy_DELETE = False

if(IF_Levy_DELETE == True):
    # Удалим посностью Levy
    del df_src["Levy"]
else:
    # ИЛИ Удалим Levy, где равен "-"
    df_src = df_src.drop(df_src[df_src["Levy"] == "-"].index)


In [ ]:
sns.set_style('whitegrid')
sns.distplot(df_src["Price"])

In [ ]:

fig,ag = plt.subplots(figsize=(15,7))
sns.distplot(   df_src[    (df_src["Price"]<75000)&(df_src["Price"]>0)   ]["Price"]   )
sns.distplot(   df_src[    (df_src["Price"]<30000)&(df_src["Price"]>850)   ]["Price"]   )
print(f"Range of Price: [{df_src['Price'].min()} - {df_src['Price'].max()}]")

In [ ]:
# Мда, создатель датасета, видимо решил приколоться...

IF_shit_DELETE = False

if(IF_shit_DELETE == True):
    # Удалим машины ниже 850 долларов
    df_src = df_src.drop(df_src[df_src["Price"] < 850].index)
else:
    # ИЛИ оставляем всё как есть и веселимся!
    pass

In [ ]:
# Удалим посностью ID
del df_src["ID"]

# Удалим null'ы
df_src = df_src.dropna()

# Удалим дубликаты, если они появились после удаления столбцов
df_src = df_src.drop_duplicates()

print(f"\nNumber of (rows, columns): {df_src.shape}")

print("\nКол-во не None значений:")
print(df_src.count())

print("\nКол-во null\'ов:")
print(df_src.isnull().sum())

## Кодирование строк + числа - это числа

In [ ]:
# Смотрим типы столбцов

pd.set_option('display.max_rows', df_src.shape[1])

nullout = '''
Должно быть:

Price                 int64
Levy                  int64
Manufacturer         cat->int
Model                cat->int
Prod. year            int64
Category             cat->int
Leather interior     cat->int
Fuel type            cat->int
Engine volume        float64
Mileage               int64
Cylinders           float64
Gear box type        cat->int
Drive wheels         cat->int
Doors                cat->int
Wheel                cat->int
Color                cat->int
Airbags               int64
'''

df_src.dtypes

In [ ]:
# Избавимся от мусора, сделаем числа числами

print(df_src["Levy"].value_counts())
df_src["Levy"] = df_src["Levy"].astype(int)

print("\n==========\n")

print(df_src["Mileage"].value_counts())
df_src["Mileage"] = df_src["Mileage"].apply(lambda x: x.split(" ")[0])
df_src["Mileage"] = df_src["Mileage"].astype(int)

print("\n==========\n")

print(df_src["Engine volume"].value_counts())
df_src["Engine volume"] = df_src["Engine volume"].apply(lambda x: x.split(" ")[0])
df_src["Engine volume"] = df_src["Engine volume"].astype(float)


In [ ]:
cat_columns = ["Manufacturer", "Model", "Category", "Leather interior", "Fuel type", "Gear box type", "Drive wheels", "Wheel", "Color", "Doors"]

for col_i in cat_columns:
    print(f"\n==========\n{col_i}")
    print(df_src[col_i].value_counts())


In [ ]:
df_nums = df_src.copy()

# Категориальные:

cat_columns_auto = ["Manufacturer", "Model", "Category", "Fuel type", "Color"]
cat_columns_man = {"Leather interior": {"No": 0, "Yes": 1}, 
                   "Gear box type": {"Manual": 0, "Automatic": 1, "Variator": 2, "Tiptronic": 3}, 
                   "Drive wheels": {"Rear": 0, "Front": 1, "4x4": 2}, 
                   "Wheel": {"Right-hand drive": 0, "Left wheel": 1}, 
                   "Doors": {"04-May": 0, "02-Mar": 1, ">5": 2}}

for col_i in cat_columns_auto:
    df_nums[col_i] = df_nums[col_i].astype('category')
    df_nums[col_i] = df_nums[col_i].cat.codes
    df_nums[col_i] = df_nums[col_i].astype(int)

for col_i in cat_columns_man:
    df_nums = df_nums.replace(cat_columns_man)

pd.set_option('display.max_rows', df_src.shape[1])
df_nums.dtypes

## Нормализация

In [ ]:
df_nums_scaled = df_nums.copy()
buff = df_nums_scaled["Price"]
df_nums_scaled = ( df_nums-df_nums.min() ) / ( df_nums.max() - df_nums.min() )
del df_nums_scaled["Price"]
df_nums_scaled.insert(0, "Price", buff)
df_nums_scaled.head(300)


# Графики

![](./imgs/which_visualization.png)


In [ ]:
num_columns = ["Price", "Levy", "Prod. year", "Engine volume", "Mileage", "Cylinders", "Airbags"]
for col_i in num_columns:
    print(f"Range of {col_i}: [{df_nums[col_i].min()} - {df_nums[col_i].max()}]")

In [ ]:
# Тепловая карта

plt.figure(figsize=(25, 13))
c= df_nums.corr()
sns.heatmap(c, cmap="BrBG", annot=True)
c

# Смотря на карту эту, невольно спрашиваешь: А почему предсказываем Price? Куда интереснее выглядит Levy

In [ ]:

num_columns = ["Levy", "Prod. year", "Engine volume", "Mileage", "Cylinders", "Airbags"]
cat_columns = ["Manufacturer", "Model", "Category", "Leather interior", "Fuel type", "Gear box type", "Drive wheels", "Wheel", "Color", "Doors"]
response_var = "Price"

df_buff = df_src[(df_src["Price"] > 850) & (df_src["Price"] < 30000)]
for cat_col_i in cat_columns:
    # Что-то не работает с производителями и моделями...
    if(cat_col_i != "Manufacturer" and cat_col_i != "Model"):
        data_wide = df_buff.pivot(columns=cat_col_i, values=response_var)
        # df_buff.pivot делает новую таблицу, где:
        # Категории из cat_col_i становятся столбцами
        # Каждая i строка имеет все NaN, кроме того столбца, где на i-ой строке в df_buff была именна эта категория,
        # и в этом столбце, где не NaN будет стоять цена на i-ой строке в df_buff
        data_wide.plot.kde(figsize = (8, 6), linewidth = 4)
        # kde - это диаграмма плотности

for num_col_i in num_columns:
    df_nums.plot.scatter(x=num_col_i, y=response_var, c="DarkBlue")
    

In [ ]:
# Гистограммы

plt.figure(figsize=(10, 10))

df_src[(df_src["Price"] > 850) & (df_src["Price"] < 30000)].hist(column="Price")
df_src[(df_src["Price"] < 30000)].hist(column="Price")

cols = df_nums.columns
for col_i in cols:
    df_nums.hist(column=col_i)


# Регрессия

In [ ]:
# coming soon...
